# CITE-seq and ASAP-seq integration using SemiLT

In this tutorial, we will illustrate the utility of scJoint step by step using CITE-seq and ASAP-seq PBMC data from control condition generated by Mimitou et al. 2020.We can use SemiLT to perform integration analysis of CITE-seq and ASAP-seq by concatenating gene expression or gene activity vectors with protein measurements. The data can be downoladed from GSE156478 or (https://github.com/SydneyBioX/scJoint/tree/main). After preprocessing, quality control and cell type annotation, the data contains 4644 cells from CITE-seq data and 4502 cells from ASAP-seq data.

In [1]:
import torch
import os
from datetime import datetime
from SemiLT.trainingprocess import Training
from SemiLT.transfer import Transfer
import time
from setting import Setting
import random
random.seed(1)
setting = Setting()

## Preparing input for SemiLT in setting

```python
if DB == "CITE-ASAP":
    self.number_of_class = 7 # Number of cell types in demo data
    self.input_size = 17219 # Number of filtered common genes and proteins between reference data and target data
    self.rna_paths = ['data_demo/adata_ref_rna.h5ad'] # GEM from reference data
    self.atac_paths = ['data_demo/adata_tar_atac.h5ad'] # GAM from target data
    self.rna_protein_paths = ['data_demo/adata_ref_adt.h5ad'] # Protein expression from reference data(filtered)
    self.atac_protein_paths = ['data_demo/adata_tar_adt.h5ad'] # Protein expression from target data(filtered)
    self.peak_paths = ['data_demo/adata_tar_peak.h5ad']
    self.atac_labels = True

    # Training setting            
    self.batch_size = 256
    self.lr = 0.008
    self.lr_decay_epoch = 20
    self.epochs = 20
    self.embedding_size = 64
    self.momentum = 0.9
    self.seed = 1
    self.checkpoint = ''
```

## View data

In [2]:
import scanpy as sc
adata_ref_rna = sc.read(setting.rna_paths[0])
print(adata_ref_rna)
adata_tar_atac = sc.read(setting.atac_paths[0])
print(adata_tar_atac)
adata_ref_adt = sc.read(setting.rna_protein_paths[0])
print(adata_ref_adt)
adata_tar_adt = sc.read(setting.atac_protein_paths[0])
print(adata_tar_adt)

AnnData object with n_obs × n_vars = 4644 × 16994
    obs: 'cell_type'
AnnData object with n_obs × n_vars = 4502 × 16994
    obs: 'cell_type'
AnnData object with n_obs × n_vars = 4644 × 225
AnnData object with n_obs × n_vars = 4502 × 225


## Running SemiLT in main.py

```python
torch.set_num_threads(1)
os.environ['OMP_NUM_THREADS'] = '1'

# initialization 
setting = Setting()    
torch.manual_seed(setting.seed)
print('Start time: ', datetime.now().strftime('%H:%M:%S'))

# Training
print('SemiLT start:')
model_stage1= Training(setting)    
for epoch in range(setting.epochs):
    print('Epoch:', epoch)
    model_stage1.train(epoch)

print('Write embeddings')
model_stage1.write_embeddings()
print('SemiLT finished: ', datetime.now().strftime('%H:%M:%S'))
```

This takes ~6 minutes using 1 thread for PyTorch.

## Label transfer in main.py

In [3]:
print('Label transfer:')
Transfer(setting, neighbors = 10, knn_rna_samples=50000)
print('Label transfer finished: ', datetime.now().strftime('%H:%M:%S'))

Label transfer:
[Label transfer] Read RNA data
[Label transfer] Read ATAC data
[Label transfer] Build Space
[Label transfer] finished
ARI:0.879897
Recall：0.954535
Precision:0.956184
F1-score：0.954619
Label transfer finished:  22:23:56
